## 1-2. Размеченный корпус.

Корпус был собран из статей, найденных в открытом доступе в google scholar. Для того, чтобы отфильтровать статьи, включающие в себя keywords как обязательный элемент, поиск был модифицирован. Статьи объединены темой психиатрических исследований по теме СДВГ.

Ключевые слова представлены в статьях внутри текста в качестве списка. Мной были вручную добавлены ключевые слова, не представленные в самих статьях, но кажущиеся важными. 

Корпус представляет собой текстовые файлы, в одном из которых находится текст статьи, в другом - ключевые слова.

In [55]:
import os
import nltk
nltk.download('stopwords')
!pip install pymorphy2
import pymorphy2
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
m = MorphAnalyzer()
from nltk.corpus import stopwords
stop = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
texts = {}
keywordss = {}
for root, dirs, files in os.walk("ncorp"):
    for name in files:
      with open(os.path.join(root, name), encoding="utf-8") as file:
        text = file.read()

        if "text" in name:
          texts[name] = text
        else:
          keywordss[name] = text

In [57]:
#сразу почистим текст: keywords превратим в списки, из самих текстов уберем табуляцию и другие теги
#также сразу приводим тексты к единому образцу - приводим к нижнему регистру, убираем стоп-слова, лемматизируем

def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    for item in lemmas:
      for element in stop:
        if item == element:
          lemmas.remove(item)
    return ' '.join(lemmas)

for key, value in keywordss.items():
  keywordss[key] = value.split('\n')
  for element in keywordss[key]:
    i = keywordss[key].index(element)
    a = element.lower()
    keywordss[key][i] = normalize_text(a)

for key, value in texts.items():
  a = value.lower() 
  texts[key] = normalize_text(a)


## 3.1. RAKE

In [23]:
!pip3 install python-rake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import RAKE
rake = RAKE.Rake(stop)

In [25]:
rake_res = {}

for key in texts:
    rake_res[key] = rake.run(texts[key], maxWords=1, minFrequency=2)

print(rake_res)

{'yaxanovtext.txt': [('сдвг', 1.0), ('правило', 1.0), ('например', 1.0), ('1', 0), ('2', 0), ('3', 0), ('4', 0), ('5', 0), ('6', 0)], 'intellecttext.txt': [('учёный', 1.0), ('sreda', 1.0), ('сдвг', 1.0), ('ребёнок', 1.0), ('самосознание', 1.0), ('эмпатия', 1.0), ('образ', 1.0), ('способ', 1.0), ('0', 0)], 'pantogamtext.txt': [('dsm-iv', 4.0), ('сдвг', 1.0), ('чтение', 1.0), ('письмо', 1.0), ('п', 1.0), ('россия', 1.0), ('москва', 1.0), ('далее', 1.0), ('тревожность', 1.0), ('пантокальцина', 1.0), ('p', 1.0), ('i', 1.0), ('moscow', 1.0), ('d', 1.0), ('болтливый', 1.0), ('гамк', 1.0), ('125993', 0), ('115184', 0), ('2017', 0), ('1', 0), ('114', 0), ('118', 0), ('2', 0), ('3', 0), ('4', 0), ('6', 0), ('7', 0)], 'logostext.txt': [('рис', 1.0), ('психолог', 1.0), ('связь', 1.0), ('например', 1.0), ('1', 0), ('10', 0)], 'comorbiditytext.txt': [('плаксивость', 1.0), ('тревога', 1.0), ('молодёжь', 1.0), ('это', 1.0), ('17', 0), ('11', 0), ('23', 0)]}


## 3.2. TextRank

In [26]:
!pip3 install summa
from summa import keywords

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
textrank_res = {}

for key in texts:
  textrank_res[key] = keywords.keywords(normalize_text(texts[key]), language='russian', additional_stopwords=stop, scores=True)

print(textrank_res)

{'yaxanovtext.txt': [('сдвг', 0.23316868089341714), ('также', 0.19640894346521673), ('человек', 0.18418294686566666), ('синдром', 0.18307152412116384), ('это состояние', 0.17007126049027685), ('взрослый', 0.1621641968173327), ('симптом', 0.12510009596864907), ('расстройство нервный', 0.12377470071513086), ('проблема', 0.12356336054336439), ('время', 0.11341748775169619), ('дать', 0.10558502376966908), ('внимание гиперактивность', 0.1034673417061482), ('трудность', 0.09977913562827208), ('нервно развитие мочь включать', 0.09476899793866109), ('ребёнок болезнь', 0.09284427706282784), ('всё', 0.09083817724677377), ('который', 0.09018315424133257), ('большой', 0.0892523926773908), ('нарушение', 0.08893120502812565), ('пациент', 0.08877330922592372), ('период жизнь', 0.08068240250597526), ('количество', 0.07749987570841743), ('месть', 0.07749443072385982), ('место', 0.07749443072385982), ('сосредоточенность', 0.075638853985173), ('сосредоточение', 0.075638853985173), ('являться', 0.07447859

## 3.3. KeyBERT

In [28]:
!pip3 install keybert
from keybert import KeyBERT
kw_model = KeyBERT('clips/mfaq')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:370: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [29]:
kb_wres = {}
kb_nres = {}

for key in texts:
  kb_wres[key] = kw_model.extract_keywords(texts[key])
  kb_nres[key] = kw_model.extract_keywords(normalize_text(texts[key]), keyphrase_ngram_range=(1, 2), stop_words=None)


## 4. Шаблоны

В ключевых словах встречаются как отдельные имена существительные и прилагательные, так и словосочетания типа Adj-Noun, Noun-Noun.

In [58]:
tags = ['NOUN', 'ADJF']

def pos_kw(n):
  tags = []
  if len(n) == 1:
    tags.append(m.parse(n)[0].tag.POS)
  else:
    a = n.split(' ')
    for item in a:
      tags.append(m.parse(item)[0].tag.POS)
  return(tags)

def pos_changer(n, m):
  new_kws = {}
  for key, value in n.items():
    item_lst = []
    for item in value:
      a = pos_kw(item[0])
      for element in a:
        if element in m:
          item_lst.append(item)
        else:
          pass
    new_kws[key] = item_lst
  return(new_kws)

filt_rake_res = pos_changer(rake_res, tags)
filt_textrank_res = pos_changer(textrank_res, tags)
filt_kb_wres = pos_changer(kb_wres, tags)
filt_kb_nres = pos_changer(kb_nres, tags)



In [59]:
print(filt_kb_nres)

{'yaxanovtext.txt': [('нервно психический', 0.9766), ('нервный', 0.9764), ('синдром дефицит', 0.9757), ('синдром дефицит', 0.9757), ('расстройство нервный', 0.9756), ('расстройство нервный', 0.9756), ('синдром сдвг', 0.9752), ('синдром сдвг', 0.9752)], 'intellecttext.txt': [('мир психолог', 0.9738), ('мир психолог', 0.9738), ('книга', 0.9724), ('by учёный', 0.9724), ('книга писатель', 0.9723), ('книга писатель', 0.9723), ('психолог феномен', 0.972), ('психолог феномен', 0.972)], 'pantogamtext.txt': [('внимание синдром', 0.9769), ('внимание синдром', 0.9769), ('неврологический психический', 0.9766), ('неврологический психический', 0.9766), ('синдром гиперактивность', 0.9754), ('синдром гиперактивность', 0.9754), ('психический', 0.9751)], 'logostext.txt': [('признак гиперактивность', 0.9769), ('признак гиперактивность', 0.9769), ('ребёнок гиперактивность', 0.9763), ('ребёнок гиперактивность', 0.9763), ('гиперактивность ребёнок', 0.9762), ('гиперактивность ребёнок', 0.9762), ('считать гип